# Resources
* [Overview of diff](https://blog.jcoglan.com/2017/02/12/the-myers-diff-algorithm-part-1/)
* [Patience diff intro](https://blog.jcoglan.com/2017/09/19/the-patience-diff-algorithm/#:~:text=What%20it%20really%20is%20is,like%20Myers%20on%20those%20pieces.&text=Patience%20diff%20splits%20this%20problem,exactly%20once%20in%20both%20versions.)

## Python Notes
dictionary insert/delete complexity: O(1)

dictionary lookup: O(n)

list insert/delete complexity: O(n)

list lookup by index: O(1)


## Steps to Find Diff
1. Read files to be compared
2. Define Slice
3. Find unique matching lines
4. Sort unique matching lines to match the most number of lines without cross matching
5. Divide into smaller slices and go to 3

### First read the two files to be compared

In [112]:
# file_A = 'a.py'
# file_B = 'b.py'

# file_A = 'c.txt'
# file_B = 'd.txt'

file_A = 'old.py'
file_B = 'new.py'

def read_files(file_A, file_B):
    with open(file_A,'r') as f:
        a_file = f.read().split('\n')

    with open(file_B,'r') as f:
        b_file = f.read().split('\n')
    
    return a_file, b_file

a_file, b_file = read_files(file_A, file_B)
a_last = len(a_file) - 1
b_last = len(b_file) - 1

read_files(file_A, file_B)

(["def build_dependency_list(deps, version_prefix=''):",
  '    """Build a list of dependency specifiers from a dependency map.',
  '',
  '    This can be used along with :py:data:`package_dependencies`,',
  '    :py:data:`npm_dependencies`, or other dependency dictionaries to build a',
  '    list of dependency specifiers for use on the command line or in',
  '    :file:`setup.py`.',
  '',
  '    Args:',
  '        deps (dict):',
  '            A dictionary of dependencies.',
  '',
  '    Returns:',
  '        list of unicode:',
  '        A list of dependency specifiers.',
  '    """',
  '    return sorted(',
  '        [',
  "            '%s%s%s' % (dep_name, version_prefix, dep_version)",
  '            for dep_name, dep_version in deps.items()',
  '        ],',
  '        key=lambda s: s.lower())',
  '',
  '',
  "def _dependency_message(message, prefix=''):",
  '    """Utility function to print and track a dependency-related message.',
  '',
  '    This will track that a message w

### Define slice class ( a subset of both file after slicing)

In [113]:
import copy
class Slice:
    def __init__(self, a_low, a_high, b_low, b_high):
        self.a_low = a_low
        self.a_high = a_high
        self.b_low = b_low
        self.b_high = b_high
    
    def is_empty(self):
        if self.a_low > self.a_high and self.b_low > self.b_high:
            return True
        else:
            return False
        
    def __str__(self):
        return 'A: ' + str((self.a_low+1, self.a_high+1)) + str(a_file[self.a_low:self.a_high+1])+\
                '\nB: ' + str((self.b_low+1, self.b_high+1)) + str(b_file[self.b_low:self.b_high+1]) + \
                '\n' + '-'*80
        

# initialize the slice, a is the whole content of file one and b is whole content of file two
start_slice = Slice(0, a_last, 0, b_last)
print(start_slice)

A: (1, 96)["def build_dependency_list(deps, version_prefix=''):", '    """Build a list of dependency specifiers from a dependency map.', '', '    This can be used along with :py:data:`package_dependencies`,', '    :py:data:`npm_dependencies`, or other dependency dictionaries to build a', '    list of dependency specifiers for use on the command line or in', '    :file:`setup.py`.', '', '    Args:', '        deps (dict):', '            A dictionary of dependencies.', '', '    Returns:', '        list of unicode:', '        A list of dependency specifiers.', '    """', '    return sorted(', '        [', "            '%s%s%s' % (dep_name, version_prefix, dep_version)", '            for dep_name, dep_version in deps.items()', '        ],', '        key=lambda s: s.lower())', '', '', "def _dependency_message(message, prefix=''):", '    """Utility function to print and track a dependency-related message.', '', '    This will track that a message was printed, allowing us to determine if', '  

### Find matches(same in content) between A and B that are unique

In [114]:
def find_unique_match(slice):
    # line content: [number of appearances in A, number of appearances in B, position in A, position in B]
    line_counts = {}
    matches = []
    
    for i in range(slice.a_low, slice.a_high + 1):
        if a_file[i] not in line_counts:
            position = i
            line_counts[a_file[i]] = [1,0,position,None]
        else:
            line_counts[a_file[i]][0] += 1
    
    for i in range(slice.b_low, slice.b_high + 1):
        # b[i] is not in line_counts indicates there's no matches
        if b_file[i] in line_counts:
            position = i
            line_counts[b_file[i]][1] += 1
            line_counts[b_file[i]][3] = position
    
    for line in line_counts:
        if line_counts[line][:2] == [1,1]:
            matches.append((line_counts[line][2],line_counts[line][3], line))
    
    return matches # (position in A, position in B)

matches = find_unique_match(start_slice)

# for match in matches:
#     print(match)
find_unique_match(start_slice)

[(0, 0, "def build_dependency_list(deps, version_prefix=''):"),
 (1, 1, '    """Build a list of dependency specifiers from a dependency map.'),
 (3, 3, '    This can be used along with :py:data:`package_dependencies`,'),
 (4,
  4,
  '    :py:data:`npm_dependencies`, or other dependency dictionaries to build a'),
 (5, 5, '    list of dependency specifiers for use on the command line or in'),
 (6, 6, '    :file:`setup.py`.'),
 (9, 9, '        deps (dict):'),
 (10, 10, '            A dictionary of dependencies.'),
 (12, 12, '    Returns:'),
 (13, 13, '        list of unicode:'),
 (14, 14, '        A list of dependency specifiers.'),
 (16, 16, '    return sorted('),
 (17, 17, '        ['),
 (18, 18, "            '%s%s%s' % (dep_name, version_prefix, dep_version)"),
 (19, 19, '            for dep_name, dep_version in deps.items()'),
 (20, 20, '        ],'),
 (21, 21, '        key=lambda s: s.lower())'),
 (24, 75, "def _dependency_message(message, prefix=''):"),
 (25,
  76,
  '    """Utility

### Use Patience Sort Algorithm to find the best way to slice code blocks

In [115]:
def patience_sort(matches):
    if len(matches) <= 1:
        return matches
        
    stacks = [matches[0]] 
    # each element in directed_matches: (position in B, position in B of the previous element on the stack)
    directed_matches = []
    longest_matches = []
    
    for match in matches[1:]:
        # since the stack is always sorted, we can use binary search to find where the new match
        # should be inserted
        stack_index = bi_search(stacks, match)
        if stack_index == -1: # front of the stacks
            stacks[0] = match
        elif stack_index == (len(stacks)-1): # end of the stacks
            stacks.append(match)
            directed_matches.append((match,stacks[stack_index]))
        elif stack_index >= 0: # middle of the stack
            stacks[stack_index+1] = match
            directed_matches.append((match,stacks[stack_index]))

    if len(directed_matches) == 0: # If only the front stack is used
        return [stacks[0]] # returns top of the front stack
    
    # finds the edge that contains the element at the end of stacks
    for match in directed_matches:
        if match[0] == stacks[-1]:
            last = match
    prev = [match for match in directed_matches if match[0] == last[1]][0]
    
    # Find the whole linked matches
    while 1:
        longest_matches.append(last[0])
        last = prev
        prev = [match for match in directed_matches if match[0] == last[1]]
        if len(prev) == 0: # if there's no prev, which means end of the linked list
            # Append the last two edges
            longest_matches.append(last[0])
            longest_matches.append(last[1])
            break
        prev = prev[0]
    
    return list(reversed(longest_matches))

patience_sort(matches)

[(0, 0, "def build_dependency_list(deps, version_prefix=''):"),
 (1, 1, '    """Build a list of dependency specifiers from a dependency map.'),
 (3, 3, '    This can be used along with :py:data:`package_dependencies`,'),
 (4,
  4,
  '    :py:data:`npm_dependencies`, or other dependency dictionaries to build a'),
 (5, 5, '    list of dependency specifiers for use on the command line or in'),
 (6, 6, '    :file:`setup.py`.'),
 (9, 9, '        deps (dict):'),
 (10, 10, '            A dictionary of dependencies.'),
 (12, 12, '    Returns:'),
 (13, 13, '        list of unicode:'),
 (14, 14, '        A list of dependency specifiers.'),
 (16, 16, '    return sorted('),
 (17, 17, '        ['),
 (18, 18, "            '%s%s%s' % (dep_name, version_prefix, dep_version)"),
 (19, 19, '            for dep_name, dep_version in deps.items()'),
 (20, 20, '        ],'),
 (21, 21, '        key=lambda s: s.lower())'),
 (46, 41, 'def dependency_error(message):'),
 (47, 42, '    """Print a dependency error.

In [116]:
def bi_search(stacks, match):
    low, high = -1, len(stacks)
    
    while low + 1 < high: 
        mid = (low + high)//2
        if stacks[mid][1] < match[1]:
            low = mid
        else:
            high = mid
    return low

### Call patience_diff recursively util the code block has only one unique line in common

In [117]:
import copy
def patience_diff(slice):
    matches = patience_sort(find_unique_match(slice))
    if len(matches) <= 1:
        return slice
    last_edge = (len(a_file),len(b_file),a_file[-1])
    matched_slices = []
    a_line = slice.a_low
    b_line = slice.b_low
    
    for match in matches:
        new_slice = Slice(a_line, match[0], b_line, match[1])
        
        a_line = match[0]+1
        b_line = match[1]+1
        
        new_slice = match_head(copy.copy(new_slice),matched_slices)
        new_slice = match_tail(copy.copy(new_slice),matched_slices)
        
        # if new_slice is not empty after removing matching head and tail
        if not new_slice.is_empty():
            matched_slices.append(patience_diff(new_slice))
    if matched_slices[-1].a_high != a_last or matched_slices[-1].b_high != b_last -1:
        last_slice = Slice(matched_slices[-1].a_high, a_last, matched_slices[-1].b_high, b_last)
        matched_slices.append(last_slice)
    return matched_slices

def match_head(slice, matched_slices):
    """find all lines that are identical at the front and append as a slice"""
    old_a_low = slice.a_low
    old_b_low = slice.b_low
    while not slice.is_empty() and a_file[slice.a_low] == b_file[slice.b_low]:
        slice.a_low += 1
        slice.b_low += 1
    if old_a_low == slice.a_low: # if there's no matching lines on head
        return slice
    else:
        match_head_slice = Slice(old_a_low, slice.a_low - 1, old_b_low, slice.b_low - 1)
        matched_slices.append(match_head_slice)
        return slice
    
def match_tail(slice, matched_slices):
    """find all lines that are identical at the end and append as a slice"""
    old_a_high = slice.a_high
    old_b_high = slice.b_high
    while not slice.is_empty() and a_file[slice.a_high] == b_file[slice.b_high]:
        slice.a_high -= 1
        slice.b_high -= 1
    if old_a_high == slice.a_high: # if there's no matching lines on tail
        return slice
    else:
        match_tail_slice = Slice(slice.a_high + 1, old_a_high, slice.b_high + 1, old_b_high)
        matched_slices.append(match_tail_slice)
        return slice
    
slices = patience_diff(start_slice)

for slice in slices:
    print(str(slice))

A: (1, 1)["def build_dependency_list(deps, version_prefix=''):"]
B: (1, 1)["def build_dependency_list(deps, version_prefix=''):"]
--------------------------------------------------------------------------------
A: (2, 2)['    """Build a list of dependency specifiers from a dependency map.']
B: (2, 2)['    """Build a list of dependency specifiers from a dependency map.']
--------------------------------------------------------------------------------
A: (3, 4)['', '    This can be used along with :py:data:`package_dependencies`,']
B: (3, 4)['', '    This can be used along with :py:data:`package_dependencies`,']
--------------------------------------------------------------------------------
A: (5, 5)['    :py:data:`npm_dependencies`, or other dependency dictionaries to build a']
B: (5, 5)['    :py:data:`npm_dependencies`, or other dependency dictionaries to build a']
--------------------------------------------------------------------------------
A: (6, 6)['    list of dependency specif